# SQL Package

Provides simple functionality to interact with a PostgreSQL server using Python classes.

**Overview of functionality:**
* Database(self, user, password, host, dbname, port)
    * properties
        * user
        * password
        * host
        * dbname
        * port
    * methods
        * create(name) x
        * connect()
        * drop(name)
* Table(self, dbname, table, schema)
    * accepts db properties
    * properties
        * connect() --> inherited
        * fetch_data(sql, con, parse_dates)
        * get_names()
        * format_names(char_dict)
        * update_names(names_dict)
        * add_columns(columns_list, type=None)
        * compare_column_order(dataframe)
        * match_columns(dataframe)
        * save_csv(data, local_path, match_column_order=True)
        * update_values(local_path, container_path)
        * update_types(types_dict)
        * close()

## Setup

In [1]:
import os
import sys
from pathlib import Path
#sys.path[0] = str(Path(__file__).resolve().parents[2]) # Set path for custom modules
import warnings
from io import StringIO

# Set path for modules
sys.path[0] = '../'

from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

# SQL libraries
import psycopg2
from src.pipeline.database import types_dict, replace_map
from src.pipeline.transform_data import create_column_full_address, geocode_latitude_longitude, split_column_lat_long
from src.toolkits.sql import format_names

# Set notebook display options
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Get project root directory
root_dir = os.path.dirname(os.getcwd())

In [2]:
# Google Maps environment variables
#load_dotenv(find_dotenv());
#GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
#GOOGLE_AGENT = os.getenv("GOOGLE_AGENT")

In [3]:
class Database():
    
    # if modulename not in sys.modules: print...
    load_dotenv(find_dotenv());
    
    def __init__(self, user=None, password=None,
                 dbname=None, host=None, port=None):
        
        # Loaded from .env if not explicit
        self.user = user if user is not None else os.getenv("POSTGRES_USER")
        self.password = password if password is not None else os.getenv("POSTGRES_PASSWORD")
        self.dbname = dbname if dbname is not None else os.getenv("POSTGRES_DB")
        self.host = host if host is not None else os.getenv("DB_HOST")
        self.port = port if port is not None else os.getenv("DB_PORT")
        
        
        # Root directory
        self._root_dir = os.path.dirname(os.getcwd())
        #sys.path[0] = str(Path(__file__).resolve().parents[2])
        
    def _connect(self):

        """
        Connects to PostgreSQL database using psycopg2 driver. Same
        arguments as psycopg2.connect().

        Params
        --------
        dbname
        user
        password
        host
        port
        connect_timeout
        """

        try:
            con = psycopg2.connect(dbname=self.dbname,
                                   user=self.user,
                                   password=self.password,
                                    host=self.host, 
                                    port=self.port,
                                  connect_timeout=3)            

        except Exception as e:
            print('Error:\n', e)
            return None


        return con
    
    @property
    def _con(self):
        try:
            con = self._connect()
            print('Connected as user "{}" to database "{}" on http://{}:{}.'.format(self.user,self.dbname,
                                                               self.host,self.port))
            con.close()
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()
        

In [4]:
db = Database()

In [5]:
class Table(Database):
    def __init__(self, user=None, password=None, dbname=None, host=None, port=None, table=None):
        super().__init__(user, password, dbname, host, port)
        
        self.table = table
        
        # Loaded from .env if not explicit
        self.user = user if user is not None else os.getenv("POSTGRES_USER")
        self.password = password if password is not None else os.getenv("POSTGRES_PASSWORD")
        self.dbname = dbname if dbname is not None else os.getenv("POSTGRES_DB")
        self.host = host if host is not None else os.getenv("DB_HOST")
        self.port = port if port is not None else os.getenv("DB_PORT")
    
    # Connect to database
    def __connect(self):
        return super(Table, self)._connect()
    
    # Check info on connection
    def __con(self):
        return super(Table, self)._con
    
    # Fetch data from sql query
    def fetch_data(self, sql, coerce_float=False, parse_dates=None):
        
        con = self.__connect()
        
        # Fetch fresh data
        data = pd.read_sql_query(sql=sql, con=con, coerce_float=coerce_float, parse_dates=parse_dates)

        # Replace None with np.nan
        data.fillna(np.nan, inplace=True)
        
        # Close db connection
        con.close()

        return data
    
    # Get names of column
    def get_names(self):
        
        # Specific query to retrieve table names
        sql = "SELECT * FROM information_schema.columns WHERE table_name = N'{}'".format(self.table)
        
        # Run query and extract
        con = self.__connect()
        data = pd.read_sql_query(sql, con)
        column_series = data['column_name']
        con.close()
    
        return column_series
    
    # Get types of columns, returns dict
    def get_types(self, as_dataframe=False):
        
        # Specific query to retrieve table names
        sql = '''SELECT column_name, 
        CASE 
            WHEN domain_name is not null then domain_name
            WHEN data_type='character varying' THEN 'varchar('||character_maximum_length||')'
            WHEN data_type='character' THEN 'char('||character_maximum_length||')'
            WHEN data_type='numeric' THEN 'numeric'
            ELSE data_type
        END AS type
        FROM information_schema.columns WHERE table_name = 'permits_raw';
        '''
        
        # Run query and extract
        con = self.__connect()
        data = pd.read_sql_query(sql, con)
        con.close()
        
        if as_dataframe:
            data['type'] = data['type'].str.upper()
            return data
        
        types_dict = dict(zip(data['column_name'], data['type'].str.upper()))
        
        return types_dict

    # Standardize column names using dictionary of character replacements
    def reformat_names(self, replace_map, update=False):
        
        series = self.get_names()
        
        def replace_chars(text):
            for oldchar, newchar in replace_map.items():
                text = text.replace(oldchar, newchar).lower()
            return text
        
        series = series.apply(replace_chars)
        
        # Update column names in db table
        def update_names(series):

            # Extract current columns in table
            old_columns = self.get_names()

            # Create list of reformatted columns to replace old columns 
            new_columns = series

            # SQL query string to change column names
            sql = 'ALTER TABLE {} '.format(self.table) + 'RENAME "{old_name}" to {new_name};'

            sql_query = []

            # Iterate through old column names and replace each with reformatted name 
            for idx, name in old_columns.iteritems():
                sql_query.append(sql.format(old_name=name, new_name=new_columns[idx]))

            # Join list to string
            sql_query = '\n'.join(sql_query)

            return sql_query  
        
        if not update:
            return series.apply(replace_chars)
        
        else:
            sql_query = update_names(series=series)
            
            # Execute query against database
            con = self.__connect()
            try:
                cur = con.cursor()
                cur.execute(sql_query)
                con.commit()
                cur.close()
                print('Updated table "{}".'.format(self.table))
            except Exception as e:
                con.rollback()
                print('Error:\n', e)
            finally:
                if con is not None:
                    con.close()
                    

    # Add new columns to database
    def add_columns(self, add_from):

        con = self.__connect()
        
        # Get names of current columns in PostgreSQL table
        current_names = self.get_names()

        # Get names of updated table not in current table
        updated_names = data.columns.tolist()
        new_names = list(set(updated_names) - set(current_names))

        # Check names list is not empty
        if not new_names:
            print("Table columns are already up to date.")
            return

        # Format strings for query
        alter_table_sql = "ALTER TABLE {db_table}\n"
        add_column_sql = "\tADD COLUMN {column} TEXT,\n"

        # Create a list and append ADD column statements
        sql_query = [alter_table_sql.format(db_table=self.table)]
        for name in new_names:
            sql_query.append(add_column_sql.format(column=name))

        # Join into one string
        sql_query = ''.join(sql_query)[:-2] + ";"

        # Execute query against database
        try:
            cur = con.cursor()
            cur.execute(sql_query)
            con.commit()
            cur.close()
            print('Updated table "{}".'.format(self.table))
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()

    # Compare order of columns in dataframe against order of columns in database                
    def compare_column_order(self, data):
        
        # Get columns from database as list
        db_columns = self.get_names().tolist()
        
        # Select columns from dataframe as list
        data_columns = data.columns.tolist()
        
        if set(data_columns) == set(db_columns):
            
            str1 = 'Dataframe columns match table "{}" '.format(self.table)
            
            if data_columns == db_columns:
                print(str1 + "and are in identical order.")
                return True
            else:
                print(str1 + "but are not in identical order.")                
                return False            
        else:
            if len(data_columns) > len(db_columns):
                print('Dataframe has columns not in table "{}":'.format(self.table))
                return list(set(data_columns) - set(db_columns))
            else:
                print('Dataframe missing columns that are in table "{}":'.format(self.table))
                return list(set(db_columns) - set(data_columns))
    
    # Rearrange the order of columns in dataframe to match order in table
    def match_column_order(self, data):
        
        # Get columns from database as list
        db_columns = self.get_names().tolist()

        # Select columns from dataframe as list
        data_columns = data.columns.tolist()
        
        if set(data_columns) == set(db_columns):
            if data_columns != db_columns:
                print('Rearranged dataframe columns to match table "{}".'.format(self.table))
                return data[db_columns]
            else:
                print('Dataframe columns already match table "{}".'.format(self.table))
                return data
        else:
            if len(data_columns) > len(db_columns):
                print('Dataframe has columns not in table "{}":'.format(self.table))
                return list(set(data_columns) - set(db_columns))
            else:
                print('Dataframe missing columns that are in table "{}":'.format(self.table))
                return list(set(db_columns) - set(data_columns))
            
    # Builds a query to update postgres from a csv file
    def update_values(self, data, id_col, columns=None, sep=','):
        
        # Test that dataframe matches table
        if not self.compare_column_order(data):
            raise IndexError("Dataframe columns do not match table.")
        
        # Fetch data types
        types_dict = self.get_types()
        
        # Append id_col to selected columns
        columns = None if not columns else [id_col] + columns
        
        # CREATE TABLE query
        tmp_table = "tmp_" + self.table

        column_names = self.get_names().tolist() if not columns else columns
        
        # Subsets types_dict by columns argument and formats into string
        types_dict = types_dict if not columns else {key:value for key, value in types_dict.items() if key in set(columns)}
        names = ',\n\t'.join(['{key} {val}'.format(key=key, val=val) for key, val in types_dict.items()])
        
        # Build queries
        sql_create_tmp_table = 'DROP TABLE IF EXISTS {};\n\n'.format(tmp_table)
        sql_create_tmp_table = sql_create_tmp_table + 'CREATE TABLE {tmp_table} (\n\t{names}\n);\n\n' \
                                .format(tmp_table=tmp_table, names=names)
           
        sql_update_query = 'UPDATE {db_table}\n'.format(db_table=self.table)
        
        sql_set = ["SET "]
        
        for name in column_names:
            set_sql = "{name} = {tmp_name},\n\t".format(name=name, tmp_name=tmp_table + '.' + name)
            sql_set.append(set_sql)
            
        sql_set = ''.join(sql_set)
        sql_set = sql_set[:-3] + "\n"
        
        sql_from = "FROM {tmp_table}\nWHERE {db_table}.{id_col} = {tmp_table}.{id_col};\n\n" \
                            .format(tmp_table=tmp_table, db_table=self.table, id_col=id_col)
        sql_drop = 'DROP TABLE {};\n'.format(tmp_table)
        
        sql_query_1 = sql_create_tmp_table
        sql_query_2 = sql_update_query + sql_set + sql_from + sql_drop

        # Preview sql query to debug
        #print(sql_query_1 + "# Copy into temp_table\ncur.copy_from(...)\n\n"+ sql_query_2)
        
        # Run update query
        data_buffer = StringIO(data.to_csv(header=False, index=False))
        con = self.__connect()
        try:
            cur = con.cursor()
            
            # Create tmp_table
            cur.execute(sql_query_1)

            # Copy into temp_table
            data_buffer.read()
            cur.copy_from(file=data_buffer, table=tmp_table, columns=columns, sep=sep)
            data_buffer.close()
            
            # Update from temp_table into table and delete temp
            cur.execute(sql_query_2)
            con.commit()
            cur.close()
            print('Updated table "{}".'.format(self.table))
        except Exception as e:
            con.rollback()
            print('Error:\n', e)
        finally:
            if con is not None:
                con.close()
                
####Update types

### Fetch Data

In [6]:
permits = Table(table="permits_raw")
data = permits.fetch_data(sql="SELECT * FROM permits_raw;")

In [7]:
permits._con

Connected as user "postgres" to database "permits" on http://localhost:5432.


In [8]:
permits.get_names()[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object

In [9]:
permits.reformat_names(replace_map, update=True)

Error:
 column "assessor_book" of relation "permits_raw" already exists



In [10]:
permits.get_names()[:3]

0      assessor_book
1      assessor_page
2    assessor_parcel
Name: column_name, dtype: object

In [11]:
permits.compare_column_order(data)

Dataframe columns match table "permits_raw" and are in identical order.


True

In [12]:
data = permits.fetch_data(sql="SELECT * FROM permits_raw;")

In [13]:
permits.compare_column_order(data)

Dataframe columns match table "permits_raw" and are in identical order.


True

In [14]:
data = create_column_full_address(data)

In [15]:
geocode_latitude_longitude(data)

Cost for geocoding 19 addresses is $0.10.
Geocoding...
19 locations were assigned coordinates.


,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_no_old_permit_no,pcis_permit_no,status,status_date,permit_type,permit_sub_type,permit_category,project_number,event_code,initiating_office,issue_date,address_start,address_fraction_start,address_end,address_fraction_end,street_direction,street_name,street_suffix,suffix_direction,unit_range_start,unit_range_end,zip_code,work_description,valuation,floor_area_la_zoning_code_definition,no_of_residential_dwelling_units,no_of_accessory_dwelling_units,no_of_stories,contractors_business_name,contractor_address,contractor_city,contractor_state,license_type,license_no,principal_first_name,principal_middle_name,principal_last_name,license_expiration_date,applicant_first_name,applicant_last_name,applicant_business_name,applicant_address_1,applicant_address_2,applicant_address_3,zone,occupancy,floor_area_la_building_code_definition,census_tract,council_district,latitude_longitude,applicant_relationship,existing_code,proposed_code,full_address
0,4317,003,***,TR 30210-C,NaN,LT 1,NaN,15044-90000-08405,Permit Finaled,09/10/2015,HVAC,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,INTERNET,08/18/2015,1823,1/2,1823,1/2,S,THAYER,AVE,NaN,NaN,NaN,90025,NaN,NaN,NaN,NaN,NaN,NaN,CONDITIONED AIRE MECHANICAL & ENGINEERING INC,18650 PARTHENIA STREET,NORTHRIDGE,CA,C20,532440,BRETT,MOORE,HOFFER,06/30/2016,BRETT,HOFFER,NaN,18650 PARTHENIA ST,NaN,"NORTHRIDGE, CA",R3-1-O,NaN,0,2671.00,5,"(34.05474, -118.42628)",Net Applicant,NaN,NaN,1823 S THAYER AVE 90025
1,5005,010,017,CHESTERFIELD SQUARE,NaN,465,16SL57806,16016-70000-02464,Permit Finaled,08/01/2017,Bldg-Alter/Repair,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,SOUTH LA,02/04/2016,2122,NaN,2122,NaN,W,54TH,ST,NaN,NaN,NaN,90062,General rehabilitation for single family dwell...,40000.00,NaN,NaN,NaN,NaN,OWNER-BUILDER,NaN,NaN,NaN,NA,0,JAVIER,NaN,TALAMANTES,NaN,JAVIER,TALAMANTES,OWNER-BUILDER,NaN,NaN,NaN,C2-1VL,NaN,NaN,2325.00,8,"(33.99307, -118.31668)",Owner-Bldr,1,NaN,2122 W 54TH ST 90062
2,5154,023,022,SUN-SET TRACT,D,13,14VN81535,14016-20000-13092,Issued,08/13/2014,Bldg-Alter/Repair,Apartment,Plan Check,NaN,NaN,VAN NUYS,08/13/2014,415,NaN,415,NaN,S,BURLINGTON,AVE,NaN,1-30,1-30,90057,PHOTOVOLTAIC SOLAR PANELS ON ROOF OF (E) APT BLDG,37000.00,NaN,NaN,NaN,NaN,PERMACITY CONSTRUCTION CORP,5570 W WASHINGTON BLVD,LOS ANGELES,CA,B,827864,JONATHAN,SAUL,PORT,11/30/2015,LINDA,MARTON,NaN,710 WILSHIRE BLVD,NaN,"SANTA MONICA, CA",R4-1,NaN,NaN,2089.04,1,"(34.06012, -118.26997)",Agent for Owner,5,NaN,415 S BURLINGTON AVE 90057
3,4404,030,010,TR 12086,NaN,2,NaN,16044-30000-09658,Permit Finaled,08/29/2016,HVAC,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,WEST LA,08/22/2016,315,NaN,315,NaN,S,OCEANO,DR,NaN,NaN,NaN,90049,NaN,NaN,NaN,NaN,NaN,NaN,E/C HEATING AND AIR CONDITION,26888 CUATRO MILPAS ST,VALENCIA,CA,C20,651051,EDY,RUDOLFO,CORDON,07/31/2018,NaN,NaN,NaN,NaN,NaN,NaN,RS-1,NaN,0,2640.00,11,"(34.05707, -118.4732)",Contractor,NaN,NaN,315 S OCEANO DR 90049
4,2646,019,011,TR 7158,NaN,11,NaN,17042-90000-31792,Permit Finaled,12/28/2017,Plumbing,1 or 2 Family Dwelling,No Plan Check,NaN,NaN,INTERNET,12/26/2017,13640,NaN,13640,NaN,W,PIERCE,ST,NaN,NaN,NaN,91331,NaN,NaN,NaN,NaN,NaN,NaN,TITANIUM POWER INC,1545 S LA CIENEGA BLVD,LOS ANGELES,CA,B,989217,DENNIS,HARUO,MIYAHIRA,12/31/2017,YONI,GHERMEZI,NaN,1545 S LA CIENEGA BLVD,NaN,"LOS ANGELES, CA",R1-1-O,NaN,0,1044.03,7,"(34.25487, -118.43002)",Net Applicant,NaN,NaN,13640 W PIERCE ST 91331
5,2219,027,052,TR 73820,NaN,52,18VN77133,17010-20000-02747,CofO Issued,04/05/2019,Bldg-New,1 or 2 Family Dwelling,Plan Check,NaN,NaN,VAN NUYS,09/21/2018,7111,NaN,7111,NaN,N,MARISA,RD,NaN,NaN,NaN,91405,"NEW SFD/GARAGE - PLAN 1A, LOT-52",196660.00,1560,1,NaN,2,OWNER-BUILDER,NaN,NaN,NaN,NA,0,NaN,NaN,NaN,NaN,DAVID,LELIE,NaN,25152 SPRINGFIELD CT,#180,"VALENCIA, CA",(T)(Q)RD2-1,NaN,1985,1278.03,6,"(34.2003503, -118.4533963)",Agent for Owner,NaN,1,7111 N MARISA RD 91405
6,2748,027,001,TR 22446,NaN,298,14LA,14041-10000-02537,Permit Finaled,10/16

In [16]:
data = split_column_lat_long(data)

In [17]:
permits.compare_column_order(data)

Dataframe has columns not in table "permits_raw":


['longitude', 'latitude', 'full_address']

In [18]:
permits.add_columns(data)

Updated table "permits_raw".


In [19]:
permits.compare_column_order(data)

Dataframe columns match table "permits_raw" but are not in identical order.


False

In [22]:
data = permits.match_column_order(data).head();

Rearranged dataframe columns to match table "permits_raw".


In [23]:
permits.compare_column_order(data)

Dataframe columns match table "permits_raw" and are in identical order.


True

In [24]:
permits.update_values(data, id_col="pcis_permit_no", columns=['assessor_book', 'latitude'])

Dataframe columns match table "permits_raw" and are in identical order.
Updated table "permits_raw".
